## Shipment and delivery delay analysis


Let's explore the **orders** and **order_items** datasets to verify the efficiency of the shipment and delivery of the platform by answering the following questions:

1. How many orders were shipped with delay?
2. What are the regions with most delays?
3. Are there any seller with high delay rate?
4. How many orders were delivered with delay?
5. How shipping delay impacts delivery performance?
6. Are there any relation between delivery delay and the distance between sellers and customers?
7. How does freight value behaves among states?
8. Conclusions

# 1. How many orders were shipped with delay?

Let's begin with the order shipment efficiency. To do so, we will consider all orders that have a date of delivering to a carrier (represented by the column "order_delivered_carrier_date").

In [1]:
import os
import pandas as pd
import mysql.connector as mysql
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from credentials import login_config, path_dir
import datetime as dt

from bokeh.plotting import figure, show
from bokeh.io import output_notebook, curdoc, push_notebook
from bokeh.models import ColumnDataSource, HoverTool, LinearAxis, Range1d, FactorRange
from bokeh.layouts import column, row
from bokeh.models.widgets import Select

output_notebook()

Loading BokehJS ...

In [2]:
# Adjusting the working directory to better manage the file calls
os.chdir(path_dir)

In [3]:
# Report information with order dispatching efficiency per month

# Creating a connection to the MySQL Database
connection = mysql.connect(**login_config)

# parsing the period of evaluation
period = ('2016-01-01 00:00:00', '2018-12-31 23:59:59')

# Loading the query
with open("2-Queries/1-Shipment_and_delivery/delayed_shipments_by_year_month.sql") as query:
    order_shipment_delay_raw = pd.read_sql_query(query.read(), params=period, con=connection)

order_shipment_delay_raw

/home/hbeltrao/Hugo/py_environments/data_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year_months,Shipped_on_schedule,Shipped_with_delay,total_shipments,Delay_Percentage,Average_Delay,Average_Shipping_time,Average_Shipping_estimate
0,201610,167.0,129.0,296,43.58,7.909,11.421,8.214
1,201611,1.0,35.0,36,97.22,28.298,37.671,9.489
2,201612,1.0,1.0,2,50.00,58.592,33.051,4.109
3,201701,598.0,128.0,726,17.63,1.994,2.676,4.559
4,201702,1424.0,295.0,1719,17.16,3.629,3.206,5.196
5,201703,2813.0,292.0,3105,9.40,5.565,3.404,7.431
6,201704,2241.0,147.0,2388,6.16,4.181,3.400,7.992
7,201705,3848.0,331.0,4179,7.92,2.938,3.346,7.768
8,201706,3339.0,304.0,3643,8.34,2.792,3.170,7.027
9,201707,3871.0,329.0,4200,7.83,1.782,3.118,6.858


It is visible that the orders from 2016 and september-2018 had the biggest delays. Apart from those months, the rest of the period had a percentage of delayed orders inferior to 20%.

Another interesting point to take is the average delay in days. This data (calculated by the difference between the shipping date and the shipping limit) show that the months with most percentages of orders delayed also had the most delay in days.
Since this data are in the edges of the dataset and have low representation (0,35% of total orders) i will opt to truncate this data for the rest of the analysis.

In [4]:
# Recalling the query using the truncated period

# parsing the period of evaluation
period = ('2017-01-01 00:00:00', '2018-08-31 23:59:59')

# Loading the query
with open("2-Queries/1-Shipment_and_delivery/delayed_shipments_by_year_month.sql") as query:
    order_shipment_delay_per_year_month = pd.read_sql_query(query.read(), params=period, con=connection)

order_shipment_delay_per_year_month

/home/hbeltrao/Hugo/py_environments/data_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year_months,Shipped_on_schedule,Shipped_with_delay,total_shipments,Delay_Percentage,Average_Delay,Average_Shipping_time,Average_Shipping_estimate
0,201701,598.0,128.0,726,17.63,1.994,2.676,4.559
1,201702,1424.0,295.0,1719,17.16,3.629,3.206,5.196
2,201703,2813.0,292.0,3105,9.40,5.565,3.404,7.431
3,201704,2241.0,147.0,2388,6.16,4.181,3.400,7.992
4,201705,3848.0,331.0,4179,7.92,2.938,3.346,7.768
5,201706,3339.0,304.0,3643,8.34,2.792,3.170,7.027
6,201707,3871.0,329.0,4200,7.83,1.782,3.118,6.858
7,201708,4582.0,376.0,4958,7.58,2.358,3.127,6.831
8,201709,4350.0,341.0,4691,7.27,2.508,3.191,7.012
9,201710,4821.0,392.0,5213,7.52,3.064,3.340,7.139


Now we can plot the graph of delays and average shipping times to use as KPIs for monitor the shipping performance

In [5]:
# Converting the year_months column to datetype to better display on graph axis
order_shipment_delay_per_year_month['year_months'] = pd.to_datetime(order_shipment_delay_per_year_month['year_months'],format='%Y%m').dt.to_period('M')

# Creating a copy of year_months column as strings to better display on the hover tooltips
order_shipment_delay_per_year_month['date'] = order_shipment_delay_per_year_month['year_months'].astype('str')


In [7]:
# Creating the interactive version of the % of orders shipped late every month

# creating the source for plotting in bokeh
graph1_source = ColumnDataSource(order_shipment_delay_per_year_month)

# Defining the figure to draw the plots
p = figure(title="Evolution of Order Dispatch Delay", x_axis_label="Year-Months", y_axis_label="Total Orders Shipped",
                plot_width=990, plot_height=400, tools="", toolbar_location=None, x_axis_type='datetime')


#  Configuring title and axis
p.title.align = 'left'
p.title.text_font_size='12pt'
p.xaxis.ticker.desired_num_ticks = order_shipment_delay_per_year_month['year_months'].nunique()
p.y_range = Range1d(start=0, end=order_shipment_delay_per_year_month['total_shipments'].max())
p.xaxis.major_label_orientation = "vertical"
p.border_fill_color = "whitesmoke"
p.min_border_left = 80


# Creating the line plot of the total shipments of each year-month
a=p.line(x='year_months', y='total_shipments', source = graph1_source,
        color= 'red', line_width=2, legend_label="Total Shipments")


# Creating the second y axis and adding it to the figure
p.extra_y_ranges = {"Percentage" : Range1d(start=0, end=order_shipment_delay_per_year_month['Delay_Percentage'].max()+1)}
p.add_layout(LinearAxis(y_range_name="Percentage", axis_label="% of orders shipped late"), 'right')


# Creating the bar plot with the percentage of orders shipped late
b=p.vbar(x='year_months', top='Delay_Percentage', source=graph1_source, width=dt.timedelta(days=15), alpha=0.5, color= 'blue', legend_label="Delay Percentage", y_range_name="Percentage")


# Adding the hover function to display graphics information
p.add_tools(HoverTool(
    tooltips=[
        ( 'date',   '@date'),
        ( 'orders shipped',  '@total_shipments orders'), # use @{ } for field names with spaces
        ( '% of orders delayed', '@Delay_Percentage %'),
    ],

    # Display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[a]
))

# Configuring y axis color to match the graph
p.yaxis[0].axis_line_color = "red"
p.yaxis[0].major_label_text_color = "red"
p.yaxis[0].major_tick_line_color = "red"
p.yaxis[0].minor_tick_line_color = "red"
p.yaxis[0].axis_label_text_color="red"

p.yaxis[1].axis_line_color = "blue"
p.yaxis[1].major_label_text_color = "blue"
p.yaxis[1].major_tick_line_color = "blue"
p.yaxis[1].minor_tick_line_color = "blue"
p.yaxis[1].axis_label_text_color="blue"

# Adding legend outside the plot
p.add_layout(p.legend[0], 'right')

show(p)


<p style="text-align: center;"> <em>Graph-1: Evolution of Order Dispatch Delay</em> </p>

The graph shows that, even with a continuous increase of shipment number alongside 2017 and stabilizing in 2018, the percentage of orders losing shipment deadlines didn't increase the same way (although it had months with spikes).
Part of the reason for this might be that the shipping process is decentralized and handled by the seller, not the platform.

The increase in December-2017 could be related to the black friday of that year, that concentrates sales on the end of November but most of these sales are processed and shipped in the beginning of December but all these assumptions need more information and a deeper analysis to be confirmed or discarded.

In [8]:
# Let's create a lineplot to visualize the trend of shippment delay and average dispatch times in days

# creating the source for plotting in bokeh
graph2_source = ColumnDataSource(order_shipment_delay_per_year_month)

# Defining the figure to draw the plots
p = figure(title="Behavior of orders shipment, shipping estimates and shipping delays", x_axis_label="Year-Months", y_axis_label="Days",
                plot_width=990, plot_height=400, tools="", toolbar_location=None, x_axis_type='datetime')


#  Configuring title and axis
p.title.align = 'left'
p.title.text_font_size='12pt'
p.xaxis.ticker.desired_num_ticks = 20
p.y_range = Range1d(start=0, end=order_shipment_delay_per_year_month['Average_Shipping_estimate'].max()+1)
p.xaxis.major_label_orientation = "vertical"
p.border_fill_color = "whitesmoke"
p.min_border_left = 80



# Creating the line plot of the average delay for each year-month
a=p.line(x='year_months', y='Average_Delay', source = graph2_source,
        color= 'blue', line_width=2, legend_label="Average Delay")

# Creating the line plot of the Average Shipping Time for each year-month
b=p.line(x='year_months', y='Average_Shipping_time', source = graph2_source,
        color= 'orange', line_width=2, legend_label="Average Shipping Time")

# Creating the line plot of the Average Shipping Estimate for each year-month
c=p.line(x='year_months', y='Average_Shipping_estimate', source = graph2_source,
        color= 'green', line_width=2, legend_label="Average Shipping Estimate")



# Creating 3 circle pĺots just to highlight the selected dots on the Hover ool
t1=p.circle(x='year_months', y='Average_Delay', source = graph2_source, size=7,
                fill_color="blue", hover_fill_color="firebrick", hover_alpha=0.7,
                line_color=None, hover_line_color="white")

t2=p.circle(x='year_months', y='Average_Shipping_time', source = graph2_source, size=7,
                fill_color="orange", hover_fill_color="firebrick", hover_alpha=0.7,
                line_color=None, hover_line_color="white")

t3=p.circle(x='year_months', y='Average_Shipping_estimate', source = graph2_source, size=7,
                fill_color="green", hover_fill_color="firebrick", hover_alpha=0.7,
                line_color=None, hover_line_color="white")


# Adding the hover function to display graphics information
p.add_tools(HoverTool(
    tooltips=[
        ( 'date',   '@date'),
        ( 'Average Delay',  '@Average_Delay days'),
        ( 'Average Shipping Time', '@Average_Shipping_time days'),
        ( 'Average Shipping Estimate', '@Average_Shipping_estimate days')
    ],

    # Display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[t1]
))


# Adding legend outside the plot
p.add_layout(p.legend[0], 'right')

show(p)



<p style="text-align: center;"> <em>Graph-2: Behavior of orders shipment, shipping estimates and shipping delays</em> </p>

We can see that the shipping time is usually quite lower than the shipping limit but when an order is delayed, that delay is quite significant (delay is counted as the number of days passed the shipping limit.), reaching over 10 days to be shipped.

Now we need to investigate if these high delays are related to regions or seller specific.

# 2. What are the regions with most delays?


Let's break the shipping behavior by state to investigate possible delay patterns and causes.

In [9]:
# Fetching the order delay data aggregated by state

# Loading the query
with open("2-Queries/1-Shipment_and_delivery/delayed_shipments_by_state_total.sql") as query:
    order_shipment_delay_per_state = pd.read_sql_query(query.read(), params=period, con=connection)

order_shipment_delay_per_state

/home/hbeltrao/Hugo/py_environments/data_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,state,Shipped_on_schedule,Shipped_with_delay,total_shipments,Delay_Percentage,Average_Delay,Average_Shipping_time,Average_Shipping_estimate
0,SE,335.0,46.0,381,12.07,3.028,3.608,6.803
1,AL,383.0,52.0,435,11.95,3.267,3.601,6.799
2,PB,528.0,68.0,596,11.41,1.880,3.632,6.790
3,ES,2015.0,226.0,2241,10.08,2.702,3.431,6.683
4,MS,732.0,82.0,814,10.07,2.085,3.216,6.699
5,MA,729.0,81.0,810,10.00,4.097,3.753,7.193
6,PR,5108.0,550.0,5658,9.72,2.702,3.304,6.581
7,RJ,13035.0,1383.0,14418,9.59,3.225,3.348,6.648
8,RN,473.0,50.0,523,9.56,2.946,3.541,6.955
9,TO,284.0,30.0,314,9.55,2.167,3.476,6.937


In [16]:
# Creating the Bokeh function to interact with the plot of shipping delay per state
# this function is necessary for bokeh to run interactive graphs using buttons on jupyter notebooks
# PS: even with this method, bokeh interactive graphics don't show on VSCode notebooks 

def order_shipment_delay_per_state_graph(doc):
     
    df = order_shipment_delay_per_state.sort_values(by='Delay_Percentage', ascending=False).copy()
    
    
    def make_plot(source):
        
        # Defining the figure to draw the plots
        plot = figure(title="Behavior of orders shipment, shipping estimates and shipping delays per state", x_axis_label="State", y_axis_label="Days",
                    plot_width=990, plot_height=400, tools="", toolbar_location=None, x_range=source.data['state'].tolist())
      

        #  Configuring title and axis
        plot.title.align = 'left'
        plot.title.text_font_size='12pt'
        plot.y_range = Range1d(start=0, end=source.data['Delay_Percentage'].max()+1)
        plot.border_fill_color = "whitesmoke"
        plot.min_border_left = 80

        # Creating the second y axis and adding it to the figure
        plot.extra_y_ranges = {"Percentage" : Range1d(start=0, end=source.data['Delay_Percentage'].max()+1)}
        plot.add_layout(LinearAxis(y_range_name="Percentage", axis_label="% of orders shipped late"), 'right')

        # Adjusting the second y-axis color to match the lineplot
        plot.yaxis[1].axis_line_color = "red"
        plot.yaxis[1].major_label_text_color = "red"
        plot.yaxis[1].major_tick_line_color = "red"
        plot.yaxis[1].minor_tick_line_color = "red"
        plot.yaxis[1].axis_label_text_color="red"

        # Creating the line plot of the average delay for each year-month
        plot.line(x='state', y='Average_Delay', source = source,
                color= 'blue', line_width=2, legend_label="Average Delay")

        # Creating the line plot of the Average Shipping Time for each year-month
        plot.line(x='state', y='Average_Shipping_time', source = source,
                color= 'orange', line_width=2, legend_label="Average Shipping Time")

        # Creating the line plot of the Average Shipping Estimate for each year-month
        plot.line(x='state', y='Average_Shipping_estimate', source = source,
                color= 'green', line_width=2, legend_label="Average Shipping Estimate")

        # Plotting the shipping delay percentage
        plot.line(x='state', y='Delay_Percentage', source = source, y_range_name="Percentage",
                color= 'red', line_width=2, legend_label="Delay Percentage")


        # Creating 4 circle pĺots just to highlight the selected dots on the Hover Tool
        t1=plot.circle(x='state', y='Average_Delay', source = source, size=7,
                        fill_color="blue", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white")

        plot.circle(x='state', y='Average_Shipping_time', source = source, size=7,
                        fill_color="orange", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white")

        plot.circle(x='state', y='Average_Shipping_estimate', source = source, size=7,
                        fill_color="green", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white")

        plot.circle(x='state', y='Delay_Percentage', source = source, size=7,
                        fill_color="red", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white")


        # Adding the hover function to display graphics information
        plot.add_tools(HoverTool(
            tooltips=[
                ( 'state',   '@state'),
                ( 'Delay Percentage', '@Delay_Percentage %'),
                ( 'Average Delay',  '@Average_Delay days'),
                ( 'Average Shipping Time', '@Average_Shipping_time days'),
                ( 'Average Shipping Estimate', '@Average_Shipping_estimate days')
            ],

            mode='vline',
            renderers=[t1]
        ))


        # Adding legend outside the plot
        plot.add_layout(plot.legend[0], 'right')

        return plot


    def update_plot(attr, old, new):
        
                
           sorter = sort_by.value
            
           new_df = order_shipment_delay_per_state.sort_values(by=sorter, ascending=False).copy()
           
           new_source = ColumnDataSource(data=new_df)
        
           plot.x_range.factors=new_source.data['state'].tolist()
            
           source.data = ColumnDataSource.from_df(new_df)
               
        
    
    sort_by = Select(title="Sort By", value="Delay_Percentage", options=['Delay_Percentage', 'Average_Delay', 'Average_Shipping_time', 'Average_Shipping_estimate'])

    source = ColumnDataSource(data=df)

    plot = make_plot(source)

    sort_by.on_change('value', update_plot)
    
    doc.add_root(column(sort_by, plot))

In [17]:
show(order_shipment_delay_per_state_graph, notebook_handle=True)

<p style="text-align: center;"> <em>Graph-3: Behavior of orders shipment, shipping estimates and shipping delays per state</em> </p>

Shipping limit and shipping time are almost flat and parallel lines, wich may indicate that the platform doesn't seem to account order origin to estimate the shipping date (Wich makes sense to guarantee that all orders are processed the same way.). 

But looking into delay percentage and average delay, the highest numbers are concentrated on north and northeast states. This could be related to distance between the seller and buyer or because these states have smaller volume of orders and thus more complex and inefficient freight logistics.

It is interesting to point that the states with highest delay percentages do not have the highest delays in days.

Following, it is presented the evolution of shipping delay statistics for each state:

In [18]:
# Fetching data from all states per year-months

# Loading the query
with open("2-Queries/1-Shipment_and_delivery/delayed_shipments_by_state_monthly.sql") as query:
    order_shipment_delay_per_state_monthly = pd.read_sql_query(query.read(), params=period, con=connection)


# Converting the year_months column to datetype to better display on graph axis
order_shipment_delay_per_state_monthly['year_months'] = pd.to_datetime(order_shipment_delay_per_state_monthly['year_months'],format='%Y%m').dt.to_period('M')

# Creating a copy of year_months column as strings to better display on the hover tooltips
order_shipment_delay_per_state_monthly['date'] = order_shipment_delay_per_state_monthly['year_months'].astype('str')

order_shipment_delay_per_state_monthly

/home/hbeltrao/Hugo/py_environments/data_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year_months,state,Shipped_on_schedule,Shipped_with_delay,total_shipments,Delay_Percentage,Average_Delay,Average_Shipping_time,Average_Shipping_estimate,date
0,2017-01,AC,4.0,0.0,4,0.00,0.000,3.531,5.500,2017-01
1,2017-01,AL,0.0,1.0,1,100.00,1.371,5.371,4.000,2017-01
2,2017-01,BA,20.0,1.0,21,4.76,1.985,1.753,5.333,2017-01
3,2017-01,CE,6.0,1.0,7,14.29,0.172,1.435,4.000,2017-01
4,2017-01,DF,12.0,1.0,13,7.69,0.019,2.451,4.000,2017-01
...,...,...,...,...,...,...,...,...,...,...
528,2018-08,RS,370.0,35.0,405,8.64,2.405,2.959,5.515,2018-08
529,2018-08,SC,235.0,22.0,257,8.56,2.608,2.820,5.501,2018-08
530,2018-08,SE,24.0,2.0,26,7.69,0.610,3.396,6.338,2018-08
531,2018-08,SP,3514.0,468.0,3982,11.75,2.734,2.877,5.066,2018-08


In [21]:
# Creating the Bokeh function to interact with the plot of shipping delay per state

# "Behavior of orders shipment, shipping estimates and shipping delays for: " state

def order_shipment_delay_per_state_by_year_month(doc):
     
    df = order_shipment_delay_per_state_monthly.copy()
    
    
    def make_plot(source, title):
        
        plot = figure(title=title, x_axis_label="Year-Months", y_axis_label="Days",
                    plot_width=990, plot_height=400, tools="", toolbar_location=None, x_axis_type='datetime')
      

        #  Configuring title and axis
        plot.title.align = 'left'
        plot.title.text_font_size='12pt'
        plot.y_range = Range1d(start=0, end=source.data['Delay_Percentage'].max()+1)
        plot.xaxis.ticker.desired_num_ticks = 20
        plot.xaxis.major_label_orientation = "vertical"
        plot.border_fill_color = "whitesmoke"
        plot.min_border_left = 80

        # Creating the second y axis and adding it to the figure
        plot.extra_y_ranges = {"Percentage" : Range1d(start=0, end=source.data['Delay_Percentage'].max()+1)}
        plot.add_layout(LinearAxis(y_range_name="Percentage", axis_label="% of orders shipped late"), 'right')

        # Adjusting the second y-axis color to match the lineplot
        plot.yaxis[1].axis_line_color = "red"
        plot.yaxis[1].major_label_text_color = "red"
        plot.yaxis[1].major_tick_line_color = "red"
        plot.yaxis[1].minor_tick_line_color = "red"
        plot.yaxis[1].axis_label_text_color="red"

        # Creating the line plot of the average delay for each year-month
        plot.line(x='year_months', y='Average_Delay', source = source,
                color= 'blue', line_width=2, legend_label="Average Delay")

        # Creating the line plot of the Average Shipping Time for each year-month
        plot.line(x='year_months', y='Average_Shipping_time', source = source,
                color= 'orange', line_width=2, legend_label="Average Shipping Time")

        # Creating the line plot of the Average Shipping Estimate for each year-month
        plot.line(x='year_months', y='Average_Shipping_estimate', source = source,
                color= 'green', line_width=2, legend_label="Average Shipping Estimate")

        # Plotting the shipping delay percentage
        plot.line(x='year_months', y='Delay_Percentage', source = source, y_range_name="Percentage",
                color= 'red', line_width=2, legend_label="Delay Percentage")


        # Creating 4 circle pĺots just to highlight the selected dots on the Hover Tool
        t1=plot.circle(x='year_months', y='Average_Delay', source = source, size=7,
                        fill_color="blue", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white")

        plot.circle(x='year_months', y='Average_Shipping_time', source = source, size=7,
                        fill_color="orange", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white")

        plot.circle(x='year_months', y='Average_Shipping_estimate', source = source, size=7,
                        fill_color="green", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white")

        plot.circle(x='year_months', y='Delay_Percentage', source = source, size=7,
                        fill_color="red", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white", y_range_name="Percentage")


        # Adding the hover function to display graphics information
        plot.add_tools(HoverTool(
            tooltips=[
                ( 'date',   '@date'),
                ( 'Delay Percentage', '@Delay_Percentage %'),
                ( 'Average Delay',  '@Average_Delay days'),
                ( 'Average Shipping Time', '@Average_Shipping_time days'),
                ( 'Average Shipping Estimate', '@Average_Shipping_estimate days')
            ],

            mode='vline',
            renderers=[t1]
        ))


        # Adding legend outside the plot
        plot.add_layout(plot.legend[0], 'right')

        return plot


    def update_plot(attr, old, new):
        
                
           state = state_select.value
            
           new_df = df[df['state']==state].copy()
           
           plot.title.text = "Behavior of orders shipment, shipping estimates and shipping delays for: " + state
                    
           source.data = ColumnDataSource.from_df(new_df)
        
           plot.extra_y_ranges['Percentage'].end = source.data['Delay_Percentage'].max()+1 
           plot.y_range.end = source.data['Delay_Percentage'].max()+1   
           
               
        
    state="SP"
    
    state_select = Select(title="State", value="SP", options=df['state'].unique().tolist())

    source = ColumnDataSource(data=df[df['state']==state])

    plot = make_plot(source, "Behavior of orders shipment, shipping estimates and shipping delays for: " + state)

    state_select.on_change('value', update_plot)
    
    doc.add_root(column(state_select, plot))

In [22]:
show(order_shipment_delay_per_state_by_year_month, notebook_handle=True)

<p style="text-align: center;"> <em>Graph-4: Behavior of orders shipment, shipping estimates and shipping delays for each state</em> </p>

Next we will explore shipping performance per seller.

# 3. Are there any seller with high delay rate?

A deeper analysis of seller data will be done in another section, but for shipping evaluation, one factor have big impact, only 1% of the sellers are responsible for overt 25% of all items sold in the period. So in order to leverage the impact of those sellers, one way is to categorize them in 3 blocks,
big, medium and small sellers, and analyze each block.

Let's define:

*  Period of data = 2017-01-01 00:00:00 and 2018-08-31 23:59:59
*  Total number of sellers = 2.951 sellers
*  Total number of items sold = 111.120 items sold

| **Seller Type**   | **Classification  Rule in sales** | **Total Sellers** | **Sellers Representation (%)** | **Total Sales** | **Sale Representation (%)** | **sales with shipping delay'** | **delay_percentage** | **Average shipping delay** |
|-------------------|-----------------------------------|-------------------|--------------------------------|-----------------|-----------------------------|--------------------------------|----------------------|----------------------------|
| **Big Seller**    | Over 500 sales                    | 29                | 0.98%                          | 29,168          | 26.25%                      | 2,331                          | 7.99%                | 1.95                       |
| **Medium Seller** | Between 5 and 500 sales           | 1,832             | 62.08%                         | 79,757          | 71.78%                      | 7,643                          | 9.58%                | 3.17                       |
| **Smal Seller**   | less than 5 sales                 | 1,090             | 36.94%                         | 2,195           | 1.97%                       | 271                            | 12.35%               | 5.05                       |
|                   | total                             | 2,951             | 100.00%                        | 111,120         | 100.00%                     | 10,245                         | 9.21%                | 2.94                       |


In [23]:
# Querying seller statistics by seller classification

# Loading the query
with open("2-Queries/1-Shipment_and_delivery/Sellers_classification_by_items_sold.sql") as query:
    seller_category_table = pd.read_sql_query(query.read(), con=connection)

seller_category_table 

/home/hbeltrao/Hugo/py_environments/data_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,seller_classification,total_sellers,seller_representation (%),total_sales,sales_representation (%),sales with shipping delay,delay_percentage,Average shipping delay
0,Big Seller,29,0.98,29168,26.25,2331.0,7.99,1.95
1,Medium Seller,1861,63.06,79847,71.86,7661.0,9.59,3.17
2,Small Seller,1061,35.95,2100,1.89,248.0,11.81,5.14


In [24]:
# Fetching the order delay data aggregated by seller

# Loading the query
with open("2-Queries/1-Shipment_and_delivery/delayed_shipments_by_seller_category_year_month.sql") as query:
    seller_data_with_category= pd.read_sql_query(query.read(), params=period, con=connection)

# Converting the year_months column to datetype to better display on graph axis
seller_data_with_category['year_months'] = pd.to_datetime(seller_data_with_category['year_months'],format='%Y%m').dt.to_period('M')

# Creating a copy of year_months column as strings to better display on the hover tooltips
seller_data_with_category['date'] = seller_data_with_category['year_months'].astype('str')

seller_data_with_category

/home/hbeltrao/Hugo/py_environments/data_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year_months,seller_classification,orders_delayed,total_orders,delay_percentage,average_delay,date
0,2017-01,Big Seller,8.0,131,6.11,1.644625,2017-01
1,2017-01,Medium Seller,111.0,565,19.65,2.078658,2017-01
2,2017-01,Small Seller,9.0,30,30.00,1.257556,2017-01
3,2017-02,Big Seller,50.0,302,16.56,3.231280,2017-02
4,2017-02,Medium Seller,226.0,1332,16.97,3.590345,2017-02
5,2017-02,Small Seller,18.0,85,21.18,5.221944,2017-02
6,2017-03,Big Seller,37.0,686,5.39,4.732081,2017-03
7,2017-03,Medium Seller,240.0,2339,10.26,5.260904,2017-03
8,2017-03,Small Seller,15.0,80,18.75,12.485400,2017-03
9,2017-04,Big Seller,20.0,565,3.54,3.908550,2017-04


In [25]:
# Plotting shipping delay information for each seller classification

def shipping_delay_per_seller_category(doc):
     
    df = seller_data_with_category.copy()
    
    
    def make_plot(source, title):
        
        plot = figure(title=title, x_axis_label="Year-Months", y_axis_label="Days",
                    plot_width=990, plot_height=400, tools="", toolbar_location=None, x_axis_type='datetime')
      

        #  Configuring title and axis
        plot.title.align = 'left'
        plot.title.text_font_size='12pt'
        plot.y_range = Range1d(start=0, end=source.data['delay_percentage'].max()+1)
        plot.xaxis.ticker.desired_num_ticks = 20
        plot.xaxis.major_label_orientation = "vertical"
        plot.border_fill_color = "whitesmoke"
        plot.min_border_left = 80

        # Creating the second y axis and adding it to the figure
        plot.extra_y_ranges = {"Percentage" : Range1d(start=0, end=source.data['delay_percentage'].max()+1)}
        plot.add_layout(LinearAxis(y_range_name="Percentage", axis_label="% of orders shipped late"), 'right')

        # Adjusting the second y-axis color to match the lineplot
        plot.yaxis[1].axis_line_color = "red"
        plot.yaxis[1].major_label_text_color = "red"
        plot.yaxis[1].major_tick_line_color = "red"
        plot.yaxis[1].minor_tick_line_color = "red"
        plot.yaxis[1].axis_label_text_color="red"

        # Creating the line plot of the average delay for each year-month
        plot.line(x='year_months', y='average_delay', source = source,
                color= 'blue', line_width=2, legend_label="Average Delay")

        # Plotting the shipping delay percentage
        plot.line(x='year_months', y='delay_percentage', source = source, y_range_name="Percentage",
                color= 'red', line_width=2, legend_label="Delay Percentage")


        # Creating 4 circle pĺots just to highlight the selected dots on the Hover Tool
        t1=plot.circle(x='year_months', y='average_delay', source = source, size=7,
                        fill_color="blue", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white")

        
        plot.circle(x='year_months', y='delay_percentage', source = source, size=7,
                        fill_color="red", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white", y_range_name="Percentage")


        # Adding the hover function to display graphics information
        plot.add_tools(HoverTool(
            tooltips=[
                ( 'date',   '@date'),
                ( 'Delay Percentage', '@delay_percentage %'),
                ( 'Average Delay',  '@average_delay days')  
            ],

            mode='vline',
            renderers=[t1]
        ))


        # Adding legend outside the plot
        plot.add_layout(plot.legend[0], 'right')

        return plot


    def update_plot(attr, old, new):
        
                
           seller_category = seller_category_select.value
            
           new_df = df[df['seller_classification']==seller_category].copy()
           
           plot.title.text = "Evolution of Shipping Delay Percentage and Time for " + seller_category
                    
           source.data = ColumnDataSource.from_df(new_df)
        
           plot.extra_y_ranges['Percentage'].end = source.data['delay_percentage'].max()+1 
           plot.y_range.end = source.data['delay_percentage'].max()+1   
           
               
        
    seller_category="Big Seller"
    
    seller_category_select = Select(title="Seller Category", value="Big Seller", options=df['seller_classification'].unique().tolist())

    source = ColumnDataSource(data=df[df['seller_classification']==seller_category])

    plot = make_plot(source, "Evolution of Shipping Delay Percentage and Time for " + seller_category)

    seller_category_select.on_change('value', update_plot)
    
    doc.add_root(column(seller_category_select, plot))

In [26]:
show(shipping_delay_per_seller_category, notebook_handle=True)

<p style="text-align: center;"> <em>Graph-5: Evolution of Shipping Delay Percentage and Time for each seller category</em> </p>

It is observable that all seller classifications had an increase in shipments delayed on the period between 2017-12 and 2018-02.
Although small sellers demonstrated a reduction tendency in their order delays, they still take the highest time to ship orders with delay.
Also, this average delay on small sellers oscillated a lot alongside the observed period.

Comparing Big and Medium sellers, both have similar average delay but medium sellers showed less volatile behavior since their numbers variated less 
during the observed period.

To finalize this analysis, let's extract delivery information and summarize the performance of the platform and their seller partners.

# 4. How many orders were delivered with delay?

Since there is no information about the carriers responsible for each delivery, we cannot assess their individual performance.
This way, we will limit the analysis to point out important statistics such as:
* Average delivery time;
* Average delivery delay (days past delivery limit);
* Delivery delay by state (state of order origin);
* The impact of seller and buyer being of different states in those statistics;

#### Average delivery time and Average delivery delay

Let's present delivery delay evolution during the observed period.

Since we are going to explore delivery statistics, we need to consider as reference date the "order_delivered_customer_date" data.
If we use "order_delivered_carrier_date" or even "order_purchase_timestamp" we will encounter orders that don't have a delivery date information
(orders with status of shipped or cancelled).

This information is important and needs to be assessed with stakeholders before beginning any analysis because the numbers may diverge from other analysis
that need to use a different date as base(like income analysis that use date of payment as reference or marketing analysis using purchase timestamp to
analyze sales on a specific marketing campaign period and etc) and sometimes those divergences may cause noise on meetings and even invalidate the report results until a proper explanation is given.

We will use the following constraints for the delivery analysis:
* Use "order_delivered_customer_date" as date reference for data aggregation
* consider only orders with "order_delivered_customer_date" filled (not null or blank)
* consider only orders delivered to customers between '2017-01-01 00:00:00' and '2018-08-31 23:59:59'
* consider only orders  with "order_status" = delivered

In [27]:
# Fetching the order delivery delay data aggregated by year-month

# Loading the query
with open("2-Queries/1-Shipment_and_delivery/delivery_delay_statistics_by_year_month.sql") as query:
    order_delivery_data= pd.read_sql_query(query.read(), params=period, con=connection)
    
# Converting the year_months column to datetype to better display on graph axis
order_delivery_data['year_months'] = pd.to_datetime(order_delivery_data['year_months'],format='%Y%m').dt.to_period('M')

# Creating a copy of year_months column as strings to better display on the hover tooltips
order_delivery_data['date'] = order_delivery_data['year_months'].astype('str')

order_delivery_data

/home/hbeltrao/Hugo/py_environments/data_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year_months,delivered_on_schedule,delivered_with_delay,total_deliveries,delay_percentage,Average_Delay,average_order_lifetime,average_delivery_duration,date
0,2017-01,326.0,0.0,326,0.00,NaN,8.545,5.886,2017-01
1,2017-02,1562.0,3.0,1565,0.19,1.169,10.452,7.482,2017-02
2,2017-03,2654.0,70.0,2724,2.57,7.300,11.874,8.546,2017-03
3,2017-04,1946.0,126.0,2072,6.08,9.323,13.032,9.573,2017-04
4,2017-05,3961.0,240.0,4201,5.71,7.579,12.892,9.456,2017-05
5,2017-06,3511.0,129.0,3640,3.54,9.465,11.657,8.588,2017-06
6,2017-07,3782.0,151.0,3933,3.84,7.801,11.260,8.108,2017-07
7,2017-08,4772.0,128.0,4900,2.61,9.116,10.795,7.658,2017-08
8,2017-09,4309.0,187.0,4496,4.16,27.060,12.308,9.151,2017-09
9,2017-10,4965.0,250.0,5215,4.79,7.384,11.674,8.359,2017-10


In [29]:
# Creation of a lineplot to visualize the trend of delivery delay and average delivery time in days

# creating the source for plotting in bokeh
graph6_source = ColumnDataSource(order_delivery_data)

# Defining the figure to draw the plots
p = figure(title="Behavior of orders delivery, delivery estimates and delivery delays", x_axis_label="Year-Months", y_axis_label="Days",
                plot_width=990, plot_height=400, tools="", toolbar_location=None, x_axis_type='datetime')


#  Configuring title and axis
p.title.align = 'left'
p.title.text_font_size='12pt'
p.xaxis.ticker.desired_num_ticks = 20
p.y_range = Range1d(start=0, end=30)
p.xaxis.major_label_orientation = "vertical"
p.border_fill_color = "whitesmoke"
p.min_border_left = 80


# Creating the second y axis and adding it to the figure
p.extra_y_ranges = {"Percentage" : Range1d(start=0, end=30)}
p.add_layout(LinearAxis(y_range_name="Percentage", axis_label="% of orders Delivered late"), 'right')

# Adjusting the second y-axis color to match the lineplot
p.yaxis[1].axis_line_color = "red"
p.yaxis[1].major_label_text_color = "red"
p.yaxis[1].major_tick_line_color = "red"
p.yaxis[1].minor_tick_line_color = "red"
p.yaxis[1].axis_label_text_color="red"


# Creating the line plot of the average delivery delay for each year-month
a=p.line(x='year_months', y='Average_Delay', source = graph6_source,
        color= 'blue', line_width=2, legend_label="Average Delay")

# Creating the line plot of the Average Delivery Time for each year-month
b=p.line(x='year_months', y='average_delivery_duration', source = graph6_source,
        color= 'orange', line_width=2, legend_label="Average Delivery Time")


# Creating the line plot of the Average Order Lifetime for each year-month
c=p.line(x='year_months', y='delay_percentage', source = graph6_source, y_range_name="Percentage",
        color= 'red', line_width=2, legend_label="Delivery Delay Percentage")




# Creating 3 circle pĺots just to highlight the selected dots on the Hover ool
t1=p.circle(x='year_months', y='Average_Delay', source = graph6_source, size=7,
                fill_color="blue", hover_fill_color="firebrick", hover_alpha=0.7,
                line_color=None, hover_line_color="white")

t2=p.circle(x='year_months', y='average_delivery_duration', source = graph6_source, size=7,
                fill_color="orange", hover_fill_color="firebrick", hover_alpha=0.7,
                line_color=None, hover_line_color="white")


t3=p.circle(x='year_months', y='delay_percentage', source = graph6_source, size=7,
                fill_color="red", hover_fill_color="firebrick", hover_alpha=0.7,
                line_color=None, hover_line_color="white")



# Adding the hover function to display graphics information
p.add_tools(HoverTool(
    tooltips=[
        ( 'date',   '@date'),
        ( 'Delay Percentage', '@delay_percentage %'),
        ( 'Average Delay',  '@Average_Delay days'),
        ( 'Average Shipping Time', '@average_delivery_duration days')
    ],

    # Display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[t1]
))


# Adding legend outside the plot
p.add_layout(p.legend[0], 'right')


show(p)

<p style="text-align: center;"> <em>Graph-6: Behavior of orders delivery, delivery estimates and delivery delays</em> </p>

Breaking the graphic in 2 parts, first we have a delivery duration with a slightly tendency of increase followed by a reduction tendency, fluctuating most of the time between 5 and 10 days. 
Second, the delay percentage also shows a tendency of increase in 2017 but followed by a strong fluctuation in 2018.

Finally, there is an abnormally high delay in 2017-09 that need to be inspected and validated, wich we will do first before get back to analyze the delay percentage fluctuation.

In [30]:
# extracting the data from 2017-09 to investigate the high delay

# Loading the query
with open("2-Queries/1-Shipment_and_delivery/order_delivered_in_2017-09.sql") as query:
    order_delivered_september_2017= pd.read_sql_query(query.read(), con=connection)
order_delivered_september_2017

/home/hbeltrao/Hugo/py_environments/data_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delay
0,ca07593549f1816d26a572e06dc1eab6,delivered,2017-02-21 23:31:27,2017-02-23 02:35:15,2017-03-08 13:47:46,2017-09-19 14:36:39,2017-03-22 00:00:00,181.6083
1,2fe324febf907e3ea3f2aa9650869fa5,delivered,2017-03-13 20:17:10,2017-03-13 20:17:10,2017-03-17 07:23:53,2017-09-19 17:00:07,2017-04-05 00:00:00,167.7083
2,285ab9426d6982034523a855f55a885e,delivered,2017-03-08 22:47:40,2017-03-08 22:47:40,2017-03-09 08:53:20,2017-09-19 14:00:04,2017-04-06 00:00:00,166.5833
3,440d0d17af552815d15a9e41abe49359,delivered,2017-03-07 23:59:51,2017-03-09 01:11:33,2017-03-15 13:00:08,2017-09-19 15:12:50,2017-04-07 00:00:00,165.6333
4,c27815f7e3dd0b926b58552628481575,delivered,2017-03-15 23:23:17,2017-03-15 23:23:17,2017-03-21 14:50:25,2017-09-19 17:14:25,2017-04-10 00:00:00,162.7181
...,...,...,...,...,...,...,...,...
170,3cba655faae78feacf27f20900c776e0,delivered,2017-09-06 20:10:17,2017-09-06 20:25:11,2017-09-12 19:12:55,2017-09-26 17:05:08,2017-09-26 00:00:00,0.7118
171,43f1558f9eec3596990432207c65c146,delivered,2017-08-28 20:03:11,2017-08-29 04:35:50,2017-08-30 19:42:06,2017-09-08 16:25:57,2017-09-08 00:00:00,0.6840
172,91575acf2a356af85894e1c3b1612e81,delivered,2017-09-06 13:07:48,2017-09-06 13:24:07,2017-09-12 13:21:23,2017-09-29 13:46:06,2017-09-29 00:00:00,0.5736
173,9413addabc2268ea995d4e968a185ab3,delivered,2017-08-14 19:45:18,2017-08-16 03:30:17,2017-08-18 15:34:43,2017-09-05 12:55:24,2017-09-05 00:00:00,0.5382


Exploring the data from September we can see that there are 48 entries with over 15 days of delay on delivery and even 21 orders with over 100 days of delay.

Analyzing those 21 entries and validating the calculations manually, the calculations proceed and all 21 orders have all information fields filled, indicating they are regular orders with no apparent error in the columns (like purchase timestamp older than delivery date and etc)
Unfortunately, there are no extra data to be used for us to understand better the reasons behind this behavior.

In [31]:
# Fetching the order delivery data aggregated by state

period = ('2017-01-01 00:00:00', '2018-08-31 23:59:59')

# Loading the query
with open("2-Queries/1-Shipment_and_delivery/delivery_delay_statistics_per_state.sql") as query:
    order_delivery_data_by_state= pd.read_sql_query(query.read(), params=period, con=connection)

/home/hbeltrao/Hugo/py_environments/data_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [35]:
# Creating the plot of delivery delay per state

def order_delivery_delay_per_state_graph(doc):
     
    df = order_delivery_data_by_state.sort_values(by='delivery_delay_percentage', ascending=False).copy()
    
    
    def make_plot(source):
        
        plot = figure(title="Behavior of orders delivery, delivery estimates and delivery delays per state", x_axis_label="State", y_axis_label="Days",
                    plot_width=990, plot_height=400, tools="", toolbar_location=None, x_range=source.data['state'].tolist())
      

        #  Configuring title and axis
        plot.title.align = 'left'
        plot.title.text_font_size='12pt'
        plot.y_range = Range1d(start=0, end=source.data['average_delay'].max()+1)
        plot.border_fill_color = "whitesmoke"
        plot.min_border_left = 80

        # Creating the second y axis and adding it to the figure
        plot.extra_y_ranges = {"Percentage" : Range1d(start=0, end=source.data['average_delay'].max()+1)}
        plot.add_layout(LinearAxis(y_range_name="Percentage", axis_label="% of orders delivered late"), 'right')

        # Adjusting the second y-axis color to match the lineplot
        plot.yaxis[1].axis_line_color = "red"
        plot.yaxis[1].major_label_text_color = "red"
        plot.yaxis[1].major_tick_line_color = "red"
        plot.yaxis[1].minor_tick_line_color = "red"
        plot.yaxis[1].axis_label_text_color="red"

        # Creating the line plot of the average delivery delay for each state
        plot.line(x='state', y='average_delay', source = source,
                color= 'blue', line_width=2, legend_label="Average Delay")

        # Creating the line plot of the Average Delivery Time for each state
        plot.line(x='state', y='average_delivery_time', source = source,
                color= 'orange', line_width=2, legend_label="Average Delivery Time")

        # Creating the line plot of the Average Delivery Estimate for each state
        plot.line(x='state', y='average_delivery_estimate', source = source,
                color= 'green', line_width=2, legend_label="Average Delivery Estimate")

        # Plotting the delivery delay percentage
        plot.line(x='state', y='delivery_delay_percentage', source = source, y_range_name="Percentage",
                color= 'red', line_width=2, legend_label="Delay Percentage")


        # Creating 4 circle pĺots just to highlight the selected dots on the Hover Tool
        t1=plot.circle(x='state', y='average_delay', source = source, size=7,
                        fill_color="blue", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white")

        plot.circle(x='state', y='average_delivery_time', source = source, size=7,
                        fill_color="orange", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white")

        plot.circle(x='state', y='average_delivery_estimate', source = source, size=7,
                        fill_color="green", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white")

        plot.circle(x='state', y='delivery_delay_percentage', source = source, size=7,
                        fill_color="red", hover_fill_color="firebrick", hover_alpha=0.7,
                        line_color=None, hover_line_color="white")


        # Adding the hover function to display graphics information
        plot.add_tools(HoverTool(
            tooltips=[
                ( 'state',   '@state'),
                ( 'Delay Percentage', '@delivery_delay_percentage %'),
                ( 'Average Delay',  '@average_delay days'),
                ( 'Average Delivery Time', '@average_delivery_time days'),
                ( 'Average Delivery Estimate', '@average_delivery_estimate days')
            ],

            mode='vline',
            renderers=[t1]
        ))


        # Adding legend outside the plot
        plot.add_layout(plot.legend[0], 'right')

        return plot


    def update_plot(attr, old, new):
        
                
           sorter = sort_by.value
            
           new_df = order_delivery_data_by_state.sort_values(by=sorter, ascending=False).copy()
           
           new_source = ColumnDataSource(data=new_df)
        
           plot.x_range.factors=new_source.data['state'].tolist()
            
           source.data = ColumnDataSource.from_df(new_df)
               
        
    
    sort_by = Select(title="Sort By", value="delivery_delay_percentage", options=['delivery_delay_percentage', 'average_delay', 'average_delivery_time', 'average_delivery_estimate'])

    source = ColumnDataSource(data=df)

    plot = make_plot(source)

    sort_by.on_change('value', update_plot)
    
    doc.add_root(column(sort_by, plot))

In [36]:
show(order_delivery_delay_per_state_graph, notebook_handle=True)

<p style="text-align: center;"> <em>Graph-7: Behavior of orders delivery, delivery estimates and delivery delays per state</em> </p>

Delivery statistics shows the highest delivery times are concentrated mostly on the northern region (wich makes sense as north is the most remote region of Brazil and most sellers are concentrated in the southeast region) followed by the northeast region.

Looking into delay information, northeast states have higher percentage of orders delivered after the deadline. Interesting point on delay is that most north region have the lowest percentage of delivery delay even having the worst logistics conditions (bad roads, huge distances, most logistics are made by plane or boat). This way, the common conclusion that "delivering orders to distant or difficult access places tend to delay more often" diverge from the data we have.

For the average delay (time past deadline for delayed deliveries), AP, RR, AM and AC have the highest numbers (they are also the most remote states of the country). This could be related to distance or difficult access but we don't have enough carrier and delivery data to test this hypotesis.

# 5. How shipping delay impacts delivery performance?

Let's see if missing shipment deadlines also increases delivery delays. For that, let's compare the percentage of delivery delay of orders shipped with and without delay.

In [38]:
# Fetching the order delivery data aggregated by year_month and with shipping delay flag

with open("2-Queries/1-Shipment_and_delivery/Ship_and_delivery_status.sql") as query:
    delivery_percentage_with_shipment_status= pd.read_sql_query(query.read(), params=period, con=connection)

# Converting the year_months column to datetype to better display on graph axis
delivery_percentage_with_shipment_status['year_months'] = pd.to_datetime(delivery_percentage_with_shipment_status['year_months'],format='%Y%m').dt.to_period('M')

# Creating a copy of year_months column as strings to better display on the hover tooltips
delivery_percentage_with_shipment_status['date'] = delivery_percentage_with_shipment_status['year_months'].astype('str')
    
delivery_percentage_with_shipment_status

/home/hbeltrao/Hugo/py_environments/data_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year_months,Ship_ok_Delivery_ok,Ship_Late_Delivery_ok,Ship_ok_Delivery_Late,Ship_Late_Delivery_Late,tot_delivered,date
0,2017-01,82.2086,17.7914,0.0000,0.0000,326,2017-01
1,2017-02,83.1949,16.6134,0.1278,0.0639,1565,2017-02
2,2017-03,87.2614,10.1689,1.2115,1.3583,2724,2017-03
3,2017-04,88.7548,5.1641,3.8610,2.2201,2072,2017-04
4,2017-05,88.5980,5.6891,3.9990,1.7139,4201,2017-05
5,2017-06,89.9451,6.5110,2.5549,0.9890,3640,2017-06
6,2017-07,89.3466,6.8141,2.5172,1.3221,3933,2017-07
7,2017-08,90.3673,7.0204,1.8980,0.7143,4900,2017-08
8,2017-09,89.9021,5.9386,3.0694,1.0899,4496,2017-09
9,2017-10,88.8207,6.3854,3.2598,1.5340,5215,2017-10


In [41]:
# Creating the plot to compare the percentage of orders shipped and delivered late

# creating the dataset to be plotted
graph8_source = ColumnDataSource(delivery_percentage_with_shipment_status)

# Defining the figure to draw the plots
p = figure(title='Orders distributed by delay on delivery and shipment', x_axis_label="Year-Months", y_axis_label="% of Total Orders",
                plot_width=990, plot_height=400, tools="", toolbar_location=None, x_axis_type='datetime')


#  Configuring title and axis
p.title.align = 'left'
p.title.text_font_size='12pt'
p.xaxis.ticker.desired_num_ticks = 20
p.y_range = Range1d(start=0, end=20)
p.xaxis.major_label_orientation = "vertical"
p.border_fill_color = "whitesmoke"
p.min_border_left = 80



# Creating the line plot of the orders shipped on schedule but delivered late
a=p.line(x='year_months', y='Ship_ok_Delivery_Late', source = graph8_source,
        color= 'blue', line_width=2, legend_label="Shipped OK Delivered late")

# Creating the line plot of the orders shipped late but delivered on schedule
b=p.line(x='year_months', y='Ship_Late_Delivery_ok', source = graph8_source,
        color= 'orange', line_width=2, legend_label="Shipped Late Delivered OK")


# Creating the line plot of the orders shipped late and delivered late
c=p.line(x='year_months', y='Ship_Late_Delivery_Late', source = graph8_source,
        color= 'red', line_width=2, legend_label="Shipped Late Delivered Late")




# Creating 3 circle pĺots just to highlight the selected dots on the Hover ool
t1=p.circle(x='year_months', y='Ship_ok_Delivery_Late', source = graph8_source, size=7,
                fill_color="blue", hover_fill_color="firebrick", hover_alpha=0.7,
                line_color=None, hover_line_color="white")

t2=p.circle(x='year_months', y='Ship_Late_Delivery_ok', source = graph8_source, size=7,
                fill_color="orange", hover_fill_color="firebrick", hover_alpha=0.7,
                line_color=None, hover_line_color="white")


t3=p.circle(x='year_months', y='Ship_Late_Delivery_Late', source = graph8_source, size=7,
                fill_color="red", hover_fill_color="firebrick", hover_alpha=0.7,
                line_color=None, hover_line_color="white")



# Adding the hover function to display graphics information
p.add_tools(HoverTool(
    tooltips=[
        ( 'date',   '@date'),
        ( 'Shipped OK Delivered late', '@Ship_ok_Delivery_Late %'),
        ( 'Shipped Late Delivered OK',  '@Ship_Late_Delivery_ok %'),
        ( 'Shipped Late Delivered Late', '@Ship_Late_Delivery_Late %')
    ],

    # Display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline',
    renderers=[t1]
))


# Adding legend outside the plot
p.add_layout(p.legend[0], 'right')


show(p)

<p style="text-align: center;"> <em>Graph-8: Orders distributed by delay on delivery and shipment</em> </p>

# 6. Is there any relation between delivery delay and the distance between seller and customer?

To answer this question, a few key data are needed:
*   Buyer and seller location (present on  this dataset)
*   Timestamp for shipping and delivery (also present)
*   descriptive data for each delivery such as transport type, presence of incidents, processing times in border crossing etc. (none of this data is available)

Since we don't have any information on delivery process and its possible problems, making an unbiased analysis becomes complex.

Over 90% of sellers are concentrated on southeast and south regions, this way, states from those regions should have the lowest numbers of delay but in fact, the delivery delay is fairly linear through states from southeast, south and northeast. This alone doesn't disprove the hypothesis.

Delay percentage also should be higher on remote regions but the north region possess the lowest percentage of orders delivered after deadlines. Wich also conflicts with the hypothesis but is not enough to disprove it.

So, having conflicting information from delivery delay percentage and delay duration indicates that is not possible to have an accurate conclusion without looking into extra data.

# 7. How does freight value behaves among states?

Let's finalize this analysis by looking into freight distribution. The most direct approach would be calculating the average freight for each origin-destination pair. We will create an interactive panel to show this information filtered per state.

Let's show the average freight value a customer would pay depending on the state he/she lives:

In [42]:
# Fetching the freight information aggregated by buyer state

period = ('2017-01-01 00:00:00', '2018-08-31 23:59:59')

# Loading the query
with open("2-Queries/1-Shipment_and_delivery/freight_value_per_buyer_state.sql") as query:
    freight_value_per_buyer_state= pd.read_sql_query(query.read(), params=period, con=connection)
freight_value_per_buyer_state

/home/hbeltrao/Hugo/py_environments/data_env/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customer_state,avg_freight,tot_orders
0,RR,44.101458,48
1,PB,42.853708,596
2,RO,41.330549,273
3,AC,40.073370,92
4,PI,39.044586,532
5,MA,38.371272,810
6,TO,37.307962,314
7,SE,36.823753,381
8,AL,35.940253,435
9,PA,35.883761,1069


In [44]:
# Plotting freight value distribution by state
graph9_source = ColumnDataSource(data=freight_value_per_buyer_state)

plot = figure(title="Average Freight Value per state", x_axis_label="State", y_axis_label="R$",
    plot_width=990, plot_height=400, tools="", toolbar_location=None, x_range=graph9_source.data['customer_state'].tolist())
      

#  Configuring title and axis
plot.title.align = 'left'
plot.title.text_font_size='12pt'
plot.y_range = Range1d(start=0, end=graph9_source.data['avg_freight'].max()+1)
plot.border_fill_color = "whitesmoke"
plot.min_border_left = 80


# Creating the line plot of the average delay for each year-month
plot.line(x='customer_state', y='avg_freight', source = graph9_source,
color= 'blue', line_width=2, legend_label="Average Freight Value")


# Creating circle pĺots just to highlight the selected dots on the Hover Tool
t1=plot.circle(x='customer_state', y='avg_freight', source = graph9_source, size=7,
fill_color="blue", hover_fill_color="firebrick", hover_alpha=0.7,
line_color=None, hover_line_color="white")


# Adding the hover function to display graphics information
plot.add_tools(HoverTool(
    tooltips=[
( 'State',   '@customer_state'),
( 'Average Freight', 'R$ @avg_freight'),
( 'Total Orders',  '@tot_orders orders')
    ],

mode='vline',
renderers=[t1]
))
    
show(plot)

<p style="text-align: center;"> <em>Graph-9: Orders distributed by delay on delivery and shipment</em> </p>

Looking to the graph, there is a significant difference between freight value from southeast and south states compared to the north and northeast states.
Since most sellers and buyers are from south and southeast, this result make sense.

# 7. Conclusions

Summarising  the key information  extracted  in the  analysis  we  have:  
  
*  The increase  in  sales  on the  platform  didn't cause an  increase  in  delay  of orders  shipments;  
*  Although  shipment  time  estimate have  decreased  alongside the  period, the real  shipment  time  didn't  followed  this  evolution;  
*  Orders of northeast  customers  tend to  miss  shipping  deadlines more often than  orders  of  customers  from other  regions:  

	* In general, orders  take, on average, the same amount of time to be shipped independently of the customer region;

	* To  avoid  bias in conclusions regarding shipping  prioritization, further analysis is needed such as verifying other prioritization  variables like order  value, quantity of order items, freight value  etc; 
	
* Small  sellers  tend to  lose  shipment  dedline  more often and the  time  after the  deadline  also is longer for  small  sellers;  
* Shipping  late doesn't reflect in  an  increase  on delivery delay;  
* The conflict  on existing  data  and  lack  of  extra  data  about delivery delay and  distance  between  seller  and  buyer unfeasibling unbiased analysis and  conclusion;
* Freight  is cheaper for south and southeast buyers;


To turn all this information in a useful tool, it could be created a recommendation system that when a client search for a product, it will return a list of sellers with the best product price alongside with freight and delivery time estimates. Usually, this information is displayed inside the product web page and not on the search list.